<a href="https://colab.research.google.com/github/explcre/AutoX/blob/master/%E2%80%9Cautox_tutorial_%E5%A4%A9%E6%B1%A0%E4%BE%9B%E5%BA%94%E9%93%BE_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

赛题链接: [2021阿里云供应链大赛](https://tianchi.aliyun.com/competition/entrance/531934/introduction)



In [ ]:
# 安装autox
!git clone https://github.com/4paradigm/AutoX.git
!pip install pytorch_tabnet
!pip install ./AutoX

Cloning into 'AutoX'...
remote: Enumerating objects: 5165, done.
remote: Counting objects: 100% (734/734), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 5165 (delta 606), reused 661 (delta 575), pack-reused 4431
Receiving objects: 100% (5165/5165), 61.63 MiB | 24.88 MiB/s, done.
Resolving deltas: 100% (2880/2880), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./AutoX
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
# tianchi_clp.zip中包含 赛事链接中下载的所有csv文件
!unzip tianchi_clp.zip

Archive:  tianchi_clp.zip
   creating: tianchi_clp/
  inflating: tianchi_clp/demand_train_A.csv  
  inflating: __MACOSX/tianchi_clp/._demand_train_A.csv  
  inflating: tianchi_clp/geo_topo.csv  
  inflating: __MACOSX/tianchi_clp/._geo_topo.csv  
  inflating: tianchi_clp/product_topo.csv  
  inflating: __MACOSX/tianchi_clp/._product_topo.csv  
  inflating: tianchi_clp/inventory_info_A.csv  
  inflating: __MACOSX/tianchi_clp/._inventory_info_A.csv  
  inflating: tianchi_clp/weight_A.csv  
  inflating: __MACOSX/tianchi_clp/._weight_A.csv  
  inflating: tianchi_clp/demand_test_A.csv  
  inflating: __MACOSX/tianchi_clp/._demand_test_A.csv  


In [ ]:
import os
import pandas as pd

## 数据预处理

In [ ]:
data_name = 'tianchi_clp'
path = f'./{data_name}'

In [ ]:
# 赛题数据demand_test_A中给了标签，我们需要将它删掉。同时我们顺便删掉无用的'Unnamed: 0'列

demand_train_A = pd.read_csv(f'{path}/demand_train_A.csv')
demand_test_A = pd.read_csv(f'{path}/demand_test_A.csv')

demand_train_A.drop('Unnamed: 0', axis=1, inplace=True)
demand_test_A.drop(['Unnamed: 0', 'qty'], axis=1, inplace=True)

In [ ]:
# 将 demand_train_A, demand_test_A 保存为train.csv, test.csv
demand_train_A.to_csv(path + '/train.csv', index = False)
demand_test_A.to_csv(path + '/test.csv', index = False)

## 导入所需的包

In [ ]:
from autox import AutoX

## 初始化AutoX类

In [ ]:
# 数据集是多表数据集，需要配置表关系
relations = [
    {
            "related_to_main_table": "true", # 是否为和主表的关系
            "left_entity": "train.csv",  # 左表名字
            "left_on": ["product"],  # 左表拼表键
            "right_entity": "product_topo.csv",  # 右表名字
            "right_on": ["product_level_2"], # 右表拼表键
            "type": "1-1" # 左表与右表的连接关系
        },  # train.csv和product_topo.csv两张表是1对1的关系，拼接键为train.csv中的product列 和 product_topo.csv中的product_level_2列
    {
            "related_to_main_table": "true", # 是否为和主表的关系
            "left_entity": "test.csv",  # 左表名字
            "left_on": ["product"],  # 左表拼表键
            "right_entity": "product_topo.csv",  # 右表名字
            "right_on": ["product_level_2"], # 右表拼表键
            "type": "1-1" # 左表与右表的连接关系
        },  # test.csv和product_topo.csv两张表是1对1的关系，拼接键为test.csv中的product列 和 product_topo.csv中的product_level_2列
    {
            "related_to_main_table": "true", # 是否为和主表的关系
            "left_entity": "train.csv",  # 左表名字
            "left_on": ["geography"],  # 左表拼表键
            "right_entity": "geo_topo.csv",  # 右表名字
            "right_on": ["geography_level_3"], # 右表拼表键
            "type": "1-1" # 左表与右表的连接关系
        },  # train.csv和geo_topo.csv两张表是1对1的关系，拼接键为train.csv中的geography列 和 geo_topo.csv中的geography_level_3列
    {
            "related_to_main_table": "true", # 是否为和主表的关系
            "left_entity": "test.csv",  # 左表名字
            "left_on": ["geography"],  # 左表拼表键
            "right_entity": "geo_topo.csv",  # 右表名字
            "right_on": ["geography_level_3"], # 右表拼表键
            "type": "1-1" # 左表与右表的连接关系
        } # test.csv和geo_topo.csv两张表是1对1的关系，拼接键为test.csv中的geography列 和 geo_topo.csv中的geography_level_3列
]

In [ ]:
autox = AutoX(target = 'qty', train_name = 'train.csv', test_name = 'test.csv', 
               id = ['unit'], path = path, time_series=True, ts_unit='D',time_col = 'ts',
               relations = relations
              )  #feature_type = feature_type,

   INFO ->  [+] read train.csv
   INFO ->  Memory usage of dataframe is 15.21 MB
   INFO ->  Memory usage after optimization is: 3.33 MB
   INFO ->  Decreased by 78.1%
   INFO ->  table = train.csv, shape = (284832, 7)
   INFO ->  [+] read test.csv
   INFO ->  Memory usage of dataframe is 2.84 MB
   INFO ->  Memory usage after optimization is: 0.45 MB
   INFO ->  Decreased by 84.3%
   INFO ->  table = test.csv, shape = (61936, 6)
   INFO ->  [+] read product_topo.csv
   INFO ->  Memory usage of dataframe is 0.00 MB
   INFO ->  Memory usage after optimization is: 0.00 MB
   INFO ->  Decreased by -166.2%
   INFO ->  table = product_topo.csv, shape = (19, 2)
   INFO ->  [+] read demand_test_A.csv
   INFO ->  Memory usage of dataframe is 3.78 MB
   INFO ->  Memory usage after optimization is: 0.92 MB
   INFO ->  Decreased by 75.7%
   INFO ->  table = demand_test_A.csv, shape = (61936, 8)
   INFO ->  [+] read demand_train_A.csv
   INFO ->  Memory usage of dataframe is 17.38 MB
   INFO ->  M

In [ ]:
sub = autox.get_submit_ts()

   INFO ->  start feature engineer
   INFO ->  feature engineer: one2M
   INFO ->  featureOne2M ops: {}
   INFO ->  ignore featureOne2M
   INFO ->  feature engineer: time
   INFO ->  featureTime ops: ['ts']
100%|██████████| 1/1 [00:00<00:00,  4.20it/s]
   INFO ->  feature engineer: ShiftTS
   INFO ->  featureShiftTS ops: ['qty']
   INFO ->  featureShiftTS lags: [98, 99, 100, 101, 105, 112, 119, 128, 196, 294]
1it [00:03,  3.67s/it]
   INFO ->  feature engineer: RollingStatTS
   INFO ->  featureRollingStatTS ops: ['qty']
   INFO ->  featureRollingStatTS windows: [105, 112, 196]
100%|██████████| 1/1 [00:08<00:00,  8.44s/it]
   INFO ->  feature engineer: ExpWeightedMean
   INFO ->  featureExpWeightedMean ops: ['qty']
   INFO ->  featureExpWeightedMean lags: [98, 99, 100, 101, 105, 112, 119, 128, 196, 294]
1it [00:04,  4.90s/it]
100%|██████████| 10/10 [00:00<00:00, 11.20it/s]
   INFO ->  label_encoder_list: ['unit', 'geography_level', 'geography', 'product_level', 'product', 'product_topo.

Training with validation
Training until validation scores don't improve for 150 rounds.
[100]	training's rmse: 5127.63	valid_1's rmse: 5165.84
[200]	training's rmse: 2285.57	valid_1's rmse: 2255.62
[300]	training's rmse: 1435.1	valid_1's rmse: 1561.21
[400]	training's rmse: 1227.52	valid_1's rmse: 1496.17
[500]	training's rmse: 1155.21	valid_1's rmse: 1492.54
[600]	training's rmse: 1116.56	valid_1's rmse: 1493.26
[700]	training's rmse: 1086.3	valid_1's rmse: 1498.03
Early stopping, best iteration is:
[550]	training's rmse: 1132.34	valid_1's rmse: 1489.08
MSE: 2217350.6524344618
Finished in 0:00:20.011857
ReTraining on all data
[100]	training's rmse: 5168.39
[200]	training's rmse: 2261.06
[300]	training's rmse: 1371.38
[400]	training's rmse: 1153.18
[500]	training's rmse: 1082.99
[600]	training's rmse: 1046.51


   INFO ->  (284832, 45)


Finished in 0:00:20.869695
Training with validation
[06:40:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:14057
Will train until validation_0-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:658.924
[200]	validation_0-rmse:661.605
Stopping. Best iteration:
[126]	validation_0-rmse:656.225

MSE: 430653.0625
Finished in 0:09:16.511224
ReTraining on all data
[06:49:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:13673.8
[100]	validation_0-rmse:673.463
[143]	validation_0-rmse:607.733


   INFO ->  feature importance
   INFO ->                        feature  feature_importance
0                unit_encoder                2107
1                     ts_year                 322
2                    ts_month                 390
3                      ts_day                 114
4               ts_weekofyear                1127
5                ts_dayofweek                   5
6                  ts_is_wknd                   0
7                  ts_quarter                  29
8           ts_is_month_start                   0
9             ts_is_month_end                   1
10          unit__qty__lag_98                 231
11          unit__qty__lag_99                  82
12         unit__qty__lag_100                  81
13         unit__qty__lag_101                 101
14         unit__qty__lag_105                 245
15         unit__qty__lag_112                 121
16         unit__qty__lag_119                 132
17         unit__qty__lag_128                 209
18     

Finished in 0:08:18.526764


In [ ]:
# 查看预测结果
sub.head()

,unit,ts,qty
0,0efede250ca3d05f9d4cc3609242d804,2021-03-02,3216.700945
1,fbb83aefc6f5d6f6bc22ae3ee757d327,2021-03-02,51.150187
2,392aaa20e70b4d7539cc7a2e09562521,2021-03-02,49.285081
3,2effa036807329a88056093fabb07ce6,2021-03-02,36168.939016
4,7dc25ea61b4d47f7de6c7a8d8d559487,2021-03-02,57862.451527


In [ ]:
# 检查预测结果和真实结果的差距
sub.rename({'qty': 'qty_pre'}, axis=1, inplace=True)
demand_test_A = pd.read_csv(f'{path}/demand_test_A.csv', usecols = ['unit','ts','qty'])

In [ ]:
analyze = demand_test_A.merge(sub, on = ['unit', 'ts'], how = 'left')

In [ ]:
analyze

,unit,ts,qty,qty_pre
0,0efede250ca3d05f9d4cc3609242d804,2021-03-02,3437.199978,3216.700945
1,fbb83aefc6f5d6f6bc22ae3ee757d327,2021-03-02,34.067925,51.150187
2,392aaa20e70b4d7539cc7a2e09562521,2021-03-02,34.856490,49.285081
3,2effa036807329a88056093fabb07ce6,2021-03-02,36677.666667,36168.939016
4,7dc25ea61b4d47f7de6c7a8d8d559487,2021-03-02,56688.333333,57862.451527
...,...,...,...,...
61931,27678e453a20126fee7e719fb4b6f933,2021-06-07,299.333333,270.521660
61932,a31b6fc53d57419631077bbfe4b241ba,2021-06-07,13695.666667,9152.683892
61933,8a54612bdaf867b47ca31e7ecc225021,2021-06-07,24504.666667,23744.845177
61934,9b8f48bacb1a63612f3a210ccc6286cc,2021-06-07,15317.428600,15288.069939


In [ ]:
# 查看mae
from sklearn.metrics import mean_absolute_error
y_true = analyze['qty']
y_pred = analyze['qty_pre']
mean_absolute_error(y_true, y_pred)

476.4757067233442

更多使用案例请参考: [autox](https://github.com/4paradigm/autox?spm=5176.21852664.0.0.5594640eeR1PoH)